### In this script we build line detector model using  Multilayer perceptron classifier in sklearn
- BY: Abdelraouf Hawash 
- DATE: 30 / 3 / 2022

### *import libraries*

In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import pickle

### important attributes and methods

In [5]:
labels = ['QR','empty','horizontal','lef3','left2','lef1','center','right1','right2','right3']

def preprocessing (img, dest_size = (20,20), dest_rang: int = 16):
    '''
    this function resize the image then makes pixels in a certain range
    it takes about 0.00035 s
    the input image should be in gray scale
    '''
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    out = (cv2.resize(img, dest_size) * (dest_rang/255)).astype('uint8')
    return out.reshape( out.shape[0] * out.shape[1] )

def show_processed_img (input, dest_size = (20,20), input_range: int = 16):
    out = (input.reshape(dest_size) * (255/input_range)).astype('uint8')
    cv2.imshow("source image", out)
    k = cv2.waitKey(0)
    cv2.destroyAllWindows()
    return k

def conv_str2list(string, labels):
    out= [0] * len(labels)
    out[labels.index(string)] = 1
    return out

def conv_list2str(Input : np.ndarray , labels):
    indexes = np.where(Input == np.amax(Input))
    return labels[indexes[0][0]]


### loading data

In [10]:
X_data = np.load('./../data/X_data.npy')
print(X_data)
print(X_data.shape)
show_processed_img(X_data[1])

Y_data = np.load('./../data/y_data.npy')
print(Y_data)
print(Y_data.shape)

Y_data = np.asarray([conv_str2list(i,labels=labels) for i in Y_data])
print(Y_data)
print(Y_data.shape)



[[11 12 12 ... 12 12 12]
 [ 5  5  5 ...  8  9  9]
 [10 10 10 ...  8  8  7]
 ...
 [14 13 14 ...  7  8  7]
 [ 5  6  5 ... 10  8  8]
 [12 13 13 ...  9  7  7]]
(4563, 400)
['lef1' 'QR' 'empty' ... 'center' 'QR' 'QR']
(4563,)
[[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]]
(4563, 10)


In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size = 0.2, random_state = 0)


### building and training model

In [13]:
model = MLPClassifier(hidden_layer_sizes=(200,200,200,200),activation="relu" ,random_state=1, max_iter=2000)


train

In [14]:
model.fit(X_data,Y_data)

MLPClassifier(hidden_layer_sizes=(200, 200, 200, 200), max_iter=2000,
              random_state=1)

In [15]:
model.score(X_data, Y_data)

0.9066403681788298

### saving model

In [24]:
pickle.dump(model, open('model', 'wb'))

### loading learned model

In [25]:
model = pickle.load(open('model', 'rb'))

### using model and calculating the time of processing

In [26]:
img = cv2.imread("./../raw_data/QR/0.jpg",0) # you must load it as gray scal image
e1 = cv2.getTickCount()
features = preprocessing(img)
predictions = model.predict([features])
e2 = cv2.getTickCount()
time = (e2 - e1)/ cv2.getTickFrequency()

print("prediction = " , conv_list2str(predictions[0], labels=labels))
print("time of processing = ",time," s") # time of processing =  0.002  s

prediction =  QR
time of processing =  0.002278015  s


### using live from camera

In [ ]:
camera = cv2.VideoCapture(0)
while (camera.isOpened):
    ret, frame = camera.read()
    cv2.imshow("camera", frame)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    features = preprocessing(img)
    predictions = model.predict([features])
    
    print(conv_list2str(predictions[0], labels= labels))
    
    if cv2.waitKey(1) == ord('q'):
        break
    
cv2.destroyAllWindows()
camera.release()
